In [1]:
import os
import pandas as pd

In [5]:
df = pd.read_csv('/Volumes/vosslabhpc/symposia/cpsy-25/data/freesurfer_7.1/extend/sub-2002/sub-2002_synth.csv')
df = df[['Intracranial-volume', 'WMH(77)']]
ratio = df['WMH(77)'] / df['Intracranial-volume']
df['Ratio'] = ratio
df

,Intracranial-volume,WMH(77),Ratio
0,1340195.2,1288.8333,0.000962
